In [23]:
import collections
import os
import io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data

'''
机器翻译
参考：https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter10_natural-language-processing/10.12_machine-translation
法语—英语数据集:https://github.com/d2l-ai/d2l-zh/blob/master/data/fr-en-small.txt
'''

PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'   #pad:填充 bos开始，eos：结束
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
# 将一个序列中所有的词记录在all_tokens中以便之后构造词典，然后在该序列后面添加PAD直到序列
# 长度变为max_seq_len，然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    all_tokens.extend(seq_tokens)
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1)
    all_seqs.append(seq_tokens)

In [41]:
# 构造词典, 并将数据中单词换成索引
def build_data(all_tokens, all_seq):
    vocab = Vocab.Vocab(collections.Counter(all_tokens), specials=[PAD, BOS, EOS])
    indices = [[vocab[w] for w in seq] for seq in all_seq]
    return vocab, torch.tensor(indices)

In [52]:
def read_data(max_seq_len):
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []
    with open('./fr-en-small.txt') as f:
        lines = f.readlines()
    for line in lines:    #从文件中逐行读取
        in_seq, out_seq = line.rstrip().split('\t')   #将每行右边多余空格去掉，并划分开输入输出句  i.e:elle est vieille . she is old .
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')   #将输入输出句各自划分成单词list i.e:['elle', 'est', 'vieille', '.'] ['she', 'is', 'old', '.']
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:
            continue  # 如果加上EOS后长于max_seq_len，则忽略掉此样本
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)
    in_vocab, in_data = build_data(in_tokens, in_seqs)
    out_vocab, out_data = build_data(out_tokens, out_seqs)
    return in_vocab, out_vocab, Data.TensorDataset(in_data, out_data)

In [55]:
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len) #in_vocab:输入词汇表  out_vocab:输出词汇表
dataset[0]

(tensor([ 5,  4, 45,  3,  2,  0,  0]), tensor([ 8,  4, 27,  3,  2,  0,  0]))

In [57]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, drop_prob=0):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):  #state:初始时刻隐藏状态(num_layers * num_directions, batch, hidden_size)
        embeddings = self.embedding(inputs.long()).permute(1, 0, 2) # 词嵌入之后将维度转换成(seq_len, batch, input_size)
        output, state = self.rnn(embeddings)
        #output(seq_len, batch, hidden_size * num_directions) state末时刻隐藏状态(num_layers * num_directions, batch, hidden_size)
        return output, state  
    
    def begin_state(self):
        return None # 隐藏态初始化为None时PyTorch会自动初始化为0

In [60]:
model = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
output, state = model(torch.zeros((4, 7)), model.begin_state())
output.shape, state.shape # GRU的state是h, 而LSTM的是一个元组(h, c)

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

In [61]:
def attention_model(input_size, attention_size):
    model = nn.Sequential(nn.Linear(input_size, attention_size, bias=False), nn.Tanh(), nn.Linear(attention_size, 1, bias=False))
    return model

def attention_forward(model, enc_states, dec_state):
    """
    enc_states: 编码器隐藏状态(时间步数, 批量大小, 隐藏单元个数)
    dec_state: 解码器隐藏状态(批量大小, 隐藏单元个数)
    """
    # 将解码器隐藏状态广播到和编码器隐藏状态形状相同后进行连结
    dec_states = dec_state.unsqueeze(dim=0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)  # 形状为(时间步数, 批量大小, 1)
    alpha = F.softmax(e, dim=0)  # 在时间步维度做softmax运算
    return (alpha * enc_states).sum(dim=0)  # 返回背景变量

In [62]:
# i.e
seq_len, batch_size, num_hiddens = 10, 4, 8
model = attention_model(2*num_hiddens, 10) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape # torch.Size([4, 8])

torch.Size([4, 8])

In [63]:
class Decoder(nn.Module):
    def __init__(slef, vocab_size, embed_size, num_hiddens, num_layers, attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, embed_size)
        # GRU的输入包含attention输出的c和实际输入, 所以尺寸是 num_hiddens+embed_size
        self.rnn = nn.GRU(num_hiddens + embed_size, num_hiddens, num_layers, dropout=drop_prob)
        self.fc = nn.Linear(num_hiddens, vocab_size)
        
    def forward(self, inputs, state):
        """
        inputs shape: (batch, )
        state shape: (num_layers, batch, num_hiddens)
        """
        c = attention_forward(self.attention, )
        embeddings = self.embedding(inputs)
        self.rnn(embeddings)
        
    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态
        return enc_state

In [ ]:
#
def batch_loss(encoder, decoder, X, Y, criterion):
    

In [21]:
#Train
embed_size, num_hiddens, num_layers = 64, 64, 2
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers, drop_prob)
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers, attention_size, drop_prob)

criterion = nn.CrossEntropyLoss()
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.01)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01)

data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
for epoch in range(5):
    for X, Y in data_iter:
        losses= []
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        loss = batch_loss(encoder, decoder, X, Y, criterion)
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        losses.append(loss.item())
    if (epoch + 1) % 10 == 0:
        print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter)))

SyntaxError: invalid syntax (<ipython-input-21-dff5deb0ce50>, line 4)

In [ ]:
#Test
def translate(encoder, decoder, ):
    

In [22]:
#评价结果
#
def belu(pred_tokens, label_tokens, k):
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label/len_pred))
    for 
    
    return score


pred_tokens = translate()
label_tokens = label_seq.split(' ')
print('bleu %.3f, predict: %s' %((bleu(pred_tokens, label_tokens, k), ' '.join(pred_tokens)))

SyntaxError: invalid syntax (<ipython-input-22-6cd9cd836dc5>, line 6)